In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt

PATH_AMI = "../data_csv/amiwea.csv"
PATH_WIND = "../data_csv/풍력발전.csv"
PATH_RE2 = "../data_csv/core4_ipynb2_reactive_ts.csv"
PATH_PR2 = "../data_csv/core4_ipynb2_proactive_ts.csv"
# PATH_M2 = "../data_csv/core4_ipynb2_metrics.csv"  # 필요 시

데이터 로드 + 시간 파싱

In [ ]:
df_ami = pd.read_csv(PATH_AMI, parse_dates=["timestamp"])
df_ami = df_ami.sort_values("timestamp").set_index("timestamp")

df_wind = pd.read_csv(PATH_WIND)
# wind 파일 컬럼: dt, MW
df_wind["dt"] = pd.to_datetime(df_wind["dt"], errors="coerce")
df_wind = df_wind.dropna(subset=["dt"]).sort_values("dt").set_index("dt")

re2 = pd.read_csv(PATH_RE2, parse_dates=["timestamp"]).sort_values("timestamp").set_index("timestamp")
pr2 = pd.read_csv(PATH_PR2, parse_dates=["timestamp"]).sort_values("timestamp").set_index("timestamp")

(df_ami.shape, df_wind.shape, re2.shape, pr2.shape,
 df_ami.index.min(), df_ami.index.max(),
 df_wind.index.min(), df_wind.index.max(),
 re2.index.min(), re2.index.max())

 2013–2014로 자르기 + 시간단위 정렬(1H)

In [ ]:
# AMI: 2013-03 ~ 2014-02 같은 범위라 했으니 우선 2013~2014로 컷
df_ami_ = df_ami.loc["2013-01-01":"2014-12-31"].copy()

# wind: 2013~2014만
df_wind_ = df_wind.loc["2013-01-01":"2014-12-31"].copy()

# 시간 단위로 맞춤
ami_hr = df_ami_[["consumption"]].resample("1H").mean()
wind_hr = df_wind_["MW"].resample("1H").mean().fillna(0.0).to_frame("generation_mw")

# ipynb2 결과는 2020이라도 상관없이 "교집합"으로 맞출 예정 (여기서 실제로 겹치는지 확인됨)
re2_hr = re2.resample("1H").mean()
pr2_hr = pr2.resample("1H").mean()

(ami_hr.shape, wind_hr.shape, re2_hr.shape, pr2_hr.shape,
 ami_hr.index.min(), ami_hr.index.max(),
 wind_hr.index.min(), wind_hr.index.max(),
 re2_hr.index.min(), re2_hr.index.max())

wind 발전 예측 생성(persistence + hour-of-day)

In [ ]:
df_gen = wind_hr.copy()
g = df_gen["generation_mw"]

pred_persist = g.shift(1)

hour_mean = g.groupby(g.index.hour).mean()
pred_hour = pd.Series(g.index.hour.map(hour_mean).values, index=g.index)

w = 0.7
df_gen["generation_pred_mw"] = (w * pred_persist + (1 - w) * pred_hour).fillna(pred_hour).clip(lower=0.0)

df_gen.head(3)

AMI + wind 병합 & net load 구성

In [ ]:
df_grid = ami_hr.join(df_gen, how="inner")

df_grid["net_load_actual"] = df_grid["consumption"] - df_grid["generation_mw"]
df_grid["net_load_pred"]   = df_grid["consumption"] - df_grid["generation_pred_mw"]

df_grid.shape, df_grid.index.min(), df_grid.index.max(), df_grid.head(2)

reactive/proactive 병합

In [ ]:
# ipynb2 결과에서 필요한 컬럼만
re2_use = re2_hr[["demand_kw", "load_adj_kw", "discharge_kw", "soc_kwh"]].add_prefix("re2_")
pr2_use = pr2_hr[["demand_kw", "load_adj_kw", "discharge_kw", "soc_kwh"]].add_prefix("pr2_")

df_all = df_grid.join(re2_use, how="inner").join(pr2_use, how="inner")

df_all.shape, df_all.index.min(), df_all.index.max(), df_all.head(2)

공통 규칙 파라미터 + 학습용 타깃 정의
	•	공통 규칙: threshold=95%, alpha_pred=0.10, alpha_ess=0.05
	•	학습 타깃(teacher): 배터리 정보가 풍부한 환경에서 나온 **proactive의 discharge를 “정답”**으로 두고,
AMI+wind(저정보)로 그 discharge를 예측하는 모델을 학습.

In [ ]:
# ✅ 공통 규칙
threshold = df_all["net_load_actual"].quantile(0.95)
alpha_pred = 0.10
alpha_ess  = 0.05
alpha_total = alpha_pred + alpha_ess

# 학습 타깃: proactive discharge (teacher)
y = df_all["pr2_discharge_kw"].copy()

# 입력 피처: AMI + wind + 예측 + 랙
X = pd.DataFrame(index=df_all.index)
X["consumption"] = df_all["consumption"]
X["generation_mw"] = df_all["generation_mw"]
X["generation_pred_mw"] = df_all["generation_pred_mw"]
X["net_load_actual"] = df_all["net_load_actual"]
X["net_load_pred"] = df_all["net_load_pred"]

# lag (정보 밀도 낮은 환경에서 흔히 쓰는 보강)
for col in ["consumption", "generation_mw", "net_load_actual"]:
    X[f"{col}_lag1"] = X[col].shift(1)
    X[f"{col}_lag2"] = X[col].shift(2)

# hour-of-day
X["hour"] = X.index.hour

# 결측 제거(shift로 생긴 NaN)
mask = X.notna().all(axis=1) & y.notna()
X = X.loc[mask]
y = y.loc[mask]

(X.shape, y.shape, threshold, alpha_total)

시계열 분할(train/test) + 모델 학습

In [ ]:
# 시계열 분할(뒤 20%를 test)
n = len(X)
split = int(n * 0.8)

X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

model = HistGradientBoostingRegressor(
    max_depth=6,
    learning_rate=0.05,
    max_iter=500,
    random_state=42
)
model.fit(X_train, y_train)

pred_test = model.predict(X_test)

mae = mean_absolute_error(y_test, pred_test)
rmse = mean_squared_error(y_test, pred_test, squared=False)
(mae, rmse)

공통 규칙 기반 baseline vs 학습모델 기반 제어 비교 (ESS 제약 포함)
	•	baseline(공통 규칙): discharge_target = alpha_total * (signal - threshold) (signal>threshold일 때)
	•	learned: 모델이 예측한 discharge를 쓰되, 동일 제약으로 클립

In [ ]:
def apply_constraints(dis_kw, power_max_kw, soc_kwh, dt_hours=1.0):
    # SOC로 가능한 최대 방전
    max_by_soc = soc_kwh / dt_hours
    return np.minimum(dis_kw, np.minimum(power_max_kw, np.maximum(max_by_soc, 0.0)))

def simulate_baseline(net_load_actual, decision_signal, threshold, alpha_total, power_max_kw, energy_max_kwh, soc_init=0.5, dt_hours=1.0):
    soc = soc_init * energy_max_kwh
    out = []
    dis_hist = []
    for actual, sig in zip(net_load_actual, decision_signal):
        if (sig > threshold) and (soc > 0):
            target = alpha_total * (sig - threshold)
        else:
            target = 0.0

        dis = min(target, power_max_kw, soc/dt_hours, actual)
        soc = max(soc - dis*dt_hours, 0.0)

        out.append(actual - dis)
        dis_hist.append(dis)
    return np.array(out), np.array(dis_hist)

# test 구간으로 평가
test = df_all.loc[X_test.index].copy()

dt_hours = 1.0

# power/energy/soc는 ipynb1에서 쓰던 값 그대로(공통 비교)
POWER_MAX = 4.0
ENERGY_MAX = 10.0
SOC_INIT = 0.5

# 1) baseline reactive: decision = net_load_actual
bl_reactive_adj, bl_reactive_dis = simulate_baseline(
    net_load_actual=test["net_load_actual"].values,
    decision_signal=test["net_load_actual"].values,
    threshold=threshold,
    alpha_total=alpha_total,
    power_max_kw=POWER_MAX,
    energy_max_kwh=ENERGY_MAX,
    soc_init=SOC_INIT,
    dt_hours=dt_hours
)

# 2) baseline proactive: decision = net_load_pred
bl_proactive_adj, bl_proactive_dis = simulate_baseline(
    net_load_actual=test["net_load_actual"].values,
    decision_signal=test["net_load_pred"].values,
    threshold=threshold,
    alpha_total=alpha_total,
    power_max_kw=POWER_MAX,
    energy_max_kwh=ENERGY_MAX,
    soc_init=SOC_INIT,
    dt_hours=dt_hours
)

# 3) learned control: 모델 예측 discharge -> 제약 적용 -> net_load_actual에서 차감
# (SOC 트레이스는 baseline과 동일하게 굴려 비교 공정성 유지)
soc = SOC_INIT * ENERGY_MAX
learn_adj = []
learn_dis = []

for actual, dis_hat in zip(test["net_load_actual"].values, pred_test):
    dis_hat = max(float(dis_hat), 0.0)
    dis = min(dis_hat, POWER_MAX, soc/dt_hours, actual)
    soc = max(soc - dis*dt_hours, 0.0)
    learn_adj.append(actual - dis)
    learn_dis.append(dis)

learn_adj = np.array(learn_adj)
learn_dis = np.array(learn_dis)

def metrics_series(x, threshold):
    x = pd.Series(x)
    ramp = x.diff().abs()
    return {
        "peak_exceed_count": int((x > threshold).sum()),
        "avg_ramp": float(ramp.mean()),
        "risky_ramp": int((ramp > ramp.quantile(0.95)).sum()),
        "exceed_energy_sum": float((x - threshold).clip(lower=0).sum())
    }

res = pd.DataFrame([
    {"scenario":"Baseline Reactive (rule)", **metrics_series(bl_reactive_adj, threshold), "ESS_throughput_kwh": float(bl_reactive_dis.sum())},
    {"scenario":"Baseline Proactive (rule)", **metrics_series(bl_proactive_adj, threshold), "ESS_throughput_kwh": float(bl_proactive_dis.sum())},
    {"scenario":"Learned Control (fit to ipynb2 proactive discharge)", **metrics_series(learn_adj, threshold), "ESS_throughput_kwh": float(learn_dis.sum())},
])

res

Power vs ESS / Reactive vs Proactive

In [ ]:
n = 300
t = test.index[:n]

plt.figure(figsize=(14,4))
plt.plot(t, test["net_load_actual"].values[:n], label="net_load_actual")
plt.plot(t, bl_reactive_adj[:n], label="after ESS (baseline reactive)")
plt.plot(t, bl_proactive_adj[:n], label="after ESS (baseline proactive)")
plt.plot(t, learn_adj[:n], label="after ESS (learned control)")
plt.axhline(threshold, linestyle="--", label="threshold (95%)")
plt.legend()
plt.title("ipynb3: Integrated Training Result (Common Rule + Learned Control)")
plt.tight_layout()
plt.show()

결과 저장 (ipynb3 산출물)

In [ ]:
res.to_csv("../data_csv/core4_ipynb3_metrics_compare.csv", index=False)

out_ts = pd.DataFrame({
    "timestamp": test.index,
    "net_load_actual": test["net_load_actual"].values,
    "baseline_reactive_adj": bl_reactive_adj,
    "baseline_proactive_adj": bl_proactive_adj,
    "learned_adj": learn_adj,
    "baseline_reactive_discharge_kw": bl_reactive_dis,
    "baseline_proactive_discharge_kw": bl_proactive_dis,
    "learned_discharge_kw": learn_dis,
})
out_ts.to_csv("../data_csv/core4_ipynb3_timeseries_compare.csv", index=False)

res, out_ts.head(3)